In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from preprocess import preprocessing
#from read_mias import *
import config
from object_detector import ObjectDetector
from custom_tensor_dataset import CustomTensorDataset
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
from imutils import paths
import pickle
import torch

import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

Implementation References:

https://pyimagesearch.com/2021/11/01/training-an-object-detector-from-scratch-in-pytorch/

https://medium.com/@sharathhebbar24/object-detection-from-scratch-5ec93520adda

https://www.kaggle.com/code/daniel601/pytorch-fasterrcnn


# Preparation

In [2]:
dir = os.getcwd()
info = pd.read_csv(dir +'/mias_info/labels.txt', sep = ' ', header = None)
info.columns = ['refno', 'tissue', 'class', 'severity','x','y','r']
info['severity'] = info['severity'].fillna('N')
info['image_id'] = info.refno.str.split('b', expand = True)[1].astype(int) -1
info.head()


,refno,tissue,class,severity,x,y,r,image_id
0,mdb001,G,CIRC,B,535,425,197.0,0
1,mdb002,G,CIRC,B,522,280,69.0,1
2,mdb003,D,NORM,N,NaN,NaN,NaN,2
3,mdb004,D,NORM,N,NaN,NaN,NaN,3
4,mdb005,F,CIRC,B,477,133,30.0,4


In [3]:
data_transform = transforms.Compose([
        transforms.ToTensor(), 
    ])

dataset = CustomTensorDataset(data_transform)


def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = torch.utils.data.DataLoader(dataset, 
                                          batch_size=4, 
                                          collate_fn=collate_fn)

In [4]:
from custom_tensor_dataset import list_files_walk
files = list_files_walk('mias_data/')

print(files)
print(len(files))

['mdb001.pgm', 'mdb002.pgm', 'mdb003.pgm', 'mdb004.pgm', 'mdb005.pgm', 'mdb006.pgm', 'mdb007.pgm', 'mdb008.pgm', 'mdb009.pgm', 'mdb010.pgm', 'mdb011.pgm', 'mdb012.pgm', 'mdb013.pgm', 'mdb014.pgm', 'mdb015.pgm', 'mdb016.pgm', 'mdb017.pgm', 'mdb018.pgm', 'mdb019.pgm', 'mdb020.pgm', 'mdb021.pgm', 'mdb022.pgm', 'mdb023.pgm', 'mdb024.pgm', 'mdb025.pgm', 'mdb026.pgm', 'mdb027.pgm', 'mdb028.pgm', 'mdb029.pgm', 'mdb030.pgm', 'mdb031.pgm', 'mdb032.pgm', 'mdb033.pgm', 'mdb034.pgm', 'mdb035.pgm', 'mdb036.pgm', 'mdb037.pgm', 'mdb038.pgm', 'mdb039.pgm', 'mdb040.pgm', 'mdb041.pgm', 'mdb042.pgm', 'mdb043.pgm', 'mdb044.pgm', 'mdb045.pgm', 'mdb046.pgm', 'mdb047.pgm', 'mdb048.pgm', 'mdb049.pgm', 'mdb050.pgm', 'mdb051.pgm', 'mdb052.pgm', 'mdb053.pgm', 'mdb054.pgm', 'mdb055.pgm', 'mdb056.pgm', 'mdb057.pgm', 'mdb058.pgm', 'mdb059.pgm', 'mdb060.pgm', 'mdb061.pgm', 'mdb062.pgm', 'mdb063.pgm', 'mdb064.pgm', 'mdb065.pgm', 'mdb066.pgm', 'mdb067.pgm', 'mdb068.pgm', 'mdb069.pgm', 'mdb070.pgm', 'mdb071.pgm', 'mdb0

In [5]:
# A set to keep track of elements that have been seen
seen = set()
# A list to store duplicates found in the input list
duplicates = []

# Iterate over each element in the list
for i in files:
    if i in seen:
        duplicates.append(i)
    else:
        seen.add(i)

print(duplicates)

[]


In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break

FileNotFoundError: [Errno 2] No such file or directory: 'mias_data\\mdb001.png'

In [7]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [8]:


model = get_model_instance_segmentation(3)



c:\Users\Student\bioe486\envs\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Student\bioe486\envs\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\Student/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:07<00:00, 21.4MB/s] 


In [11]:
num_epochs = 1
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(data_loader)

for epoch in range(num_epochs):
    model.train()
    i = 0    
    epoch_loss = 0
    for imgs, annotations in data_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model([imgs[0]], [annotations[0]])
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 
#         print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')
        epoch_loss += losses
    print(epoch_loss)

IndexError: single positional indexer is out-of-bounds

In [6]:
le = LabelEncoder()
labels = le.fit_transform(labels)

In [7]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(data, labels, bboxes, imgPaths, test_size=0.20, random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]


ValueError: Found input variables with inconsistent numbers of samples: [323, 332, 332, 323]

In [11]:
# convert NumPy arrays to PyTorch tensors
(trainImages, testImages) = torch.tensor(trainImages),\
	torch.tensor(testImages)
(trainLabels, testLabels) = torch.tensor(trainLabels),\
	torch.tensor(testLabels)
(trainBBoxes, testBBoxes) = torch.tensor(trainBBoxes),\
	torch.tensor(testBBoxes)
# define normalization transforms
transforms = transforms.Compose([
	transforms.ToPILImage(),
	transforms.ToTensor(),
	transforms.Normalize(mean=config.MEAN, std=config.STD)
])

ValueError: expected sequence of length 1 at dim 1 (got 2)

In [ ]:
# convert NumPy arrays to PyTorch datasets
trainDS = CustomTensorDataset((trainImages, trainLabels, trainBBoxes),
	transforms=transforms)
testDS = CustomTensorDataset((testImages, testLabels, testBBoxes),
	transforms=transforms)
print("[INFO] total training samples: {}...".format(len(trainDS)))
print("[INFO] total test samples: {}...".format(len(testDS)))


In [ ]:
# write the testing image paths to disk so that we can use then
# when evaluating/testing our object detector
print("[INFO] saving testing image paths...")
f = open(config.TEST_PATHS, "w")
f.write("\n".join(testPaths))
f.close()

Using Resnet50 pretrained weights : change as required

In [ ]:
# load the ResNet50 network
resnet = resnet50(pretrained=True)
# freeze all ResNet50 layers so they will *not* be updated during the
# training process
for param in resnet.parameters():
	param.requires_grad = False

Create object detector model and defines loss functions

In [ ]:
# create our custom object detector model and flash it to the current
# device
objectDetector = ObjectDetector(resnet, len(le.classes_))
objectDetector = objectDetector.to(config.DEVICE)
# define our loss functions
classLossFunc = CrossEntropyLoss()
bboxLossFunc = MSELoss()
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(objectDetector.parameters(), lr=config.INIT_LR)
print(objectDetector)


# Training

`config` file currently set for

20 epochs

batch size 32

change as necessary in the `config` before running

In [ ]:
from train_det import train
H = train(objectDetector, bboxLossFunc, classLossFunc, trainDS, testDS, opt)

In [ ]:
# serialize the model to disk
print("[INFO] saving object detector model...")
torch.save(objectDetector, config.MODEL_PATH)
# serialize the label encoder to disk
print("[INFO] saving label encoder...")
f = open(config.LE_PATH, "wb")
f.write(pickle.dumps(le))
f.close()
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["total_train_loss"], label="total_train_loss")
plt.plot(H["total_val_loss"], label="total_val_loss")
plt.plot(H["train_class_acc"], label="train_class_acc")
plt.plot(H["val_class_acc"], label="val_class_acc")
plt.title("Total Training Loss and Classification Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
# save the training plot
plotPath = os.path.sep.join([config.PLOTS_PATH, "training.png"])
plt.savefig(plotPath)

# Predictions

In [ ]:
# USAGE
# python predict.py --input dataset/images/face/image_0131.jpg
# import the necessary packages
from torchvision import transforms
import mimetypes
import argparse
import imutils
import pickle
import torch
import cv2
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--input", required=True,
	help="path to input image/text file of image paths")
args = vars(ap.parse_args())

In [ ]:
# determine the input file type, but assume that we're working with
# single input image
filetype = mimetypes.guess_type(args["input"])[0]
imagePaths = [args["input"]]
# if the file type is a text file, then we need to process *multiple*
# images
if "text/plain" == filetype:
	# load the image paths in our testing file
	imagePaths = open(args["input"]).read().strip().split("\n")

In [ ]:
# load our object detector, set it evaluation mode, and label
# encoder from disk
print("[INFO] loading object detector...")
model = torch.load(config.MODEL_PATH).to(config.DEVICE)
model.eval()
le = pickle.loads(open(config.LE_PATH, "rb").read())
# define normalization transforms
transforms = transforms.Compose([
	transforms.ToPILImage(),
	transforms.ToTensor(),
	transforms.Normalize(mean=config.MEAN, std=config.STD)
])

In [ ]:
# loop over the images that we'll be testing using our bounding box
# regression model
for imagePath in imagePaths:
	# load the image, copy it, swap its colors channels, resize it, and
	# bring its channel dimension forward
	image = cv2.imread(imagePath)
	orig = image.copy()
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	image = image.transpose((2, 0, 1))
	# convert image to PyTorch tensor, normalize it, flash it to the
	# current device, and add a batch dimension
	image = torch.from_numpy(image)
	image = transforms(image).to(config.DEVICE)
	image = image.unsqueeze(0)
		# predict the bounding box of the object along with the class
	# label
	(boxPreds, labelPreds) = model(image)
	(startX, startY, endX, endY) = boxPreds[0]
	# determine the class label with the largest predicted
	# probability
	labelPreds = torch.nn.Softmax(dim=-1)(labelPreds)
	i = labelPreds.argmax(dim=-1).cpu()
	label = le.inverse_transform(i)[0]
	
	# resize the original image such that it fits on our screen, and
	# grab its dimensions
	orig = imutils.resize(orig, width=600)
	(h, w) = orig.shape[:2]
	# scale the predicted bounding box coordinates based on the image
	# dimensions
	startX = int(startX * w)
	startY = int(startY * h)
	endX = int(endX * w)
	endY = int(endY * h)
	# draw the predicted bounding box and class label on the image
	y = startY - 10 if startY - 10 > 10 else startY + 10
	cv2.putText(orig, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX,
		0.65, (0, 255, 0), 2)
	cv2.rectangle(orig, (startX, startY), (endX, endY),
		(0, 255, 0), 2)
	# show the output image 
	cv2.imshow("Output", orig)
	cv2.waitKey(0)